In [5]:
import pandas as pd
moreDate_data = pd.read_csv("./combined_results_deduplicated_triangle.csv")

In [13]:
import pandas as pd
from datetime import timedelta

# 读取前100行（仅测试）
moreDate_data = pd.read_csv("./combined_results_deduplicated_triangle.csv", engine="python")

# 转换 Date 列为 datetime 类型
moreDate_data['Date'] = pd.to_datetime(moreDate_data['Date'])

# 使用原始 Time 列（注意：它可能是字符串，需要转为时间类型）
moreDate_data['Time'] = pd.to_datetime(moreDate_data['Time'], format="%H:%M:%S").dt.time

# 提取周几
moreDate_data['Weekday'] = moreDate_data['Date'].dt.weekday  # 0=Monday, ..., 6=Sunday

# 聚合：按 Site_ID、Weekday 和 Time（分钟粒度）计算平均值
agg_df = (
    moreDate_data
    .groupby(['Site_ID', 'Weekday', 'Time', 'Bound_Category'])['MeltedValue']
    .mean()
    .reset_index()
)

# 用 Weekday 和 Time 创建对齐时间（统一设定在 2023 年第一周）
first_monday_2023 = pd.to_datetime("2023-01-02")

agg_df['Aligned_Datetime'] = agg_df.apply(
    lambda row: pd.Timestamp(first_monday_2023 + timedelta(days=row['Weekday']))
                .replace(hour=row['Time'].hour, minute=row['Time'].minute),
    axis=1
)

# 添加排序辅助列
agg_df['Time_Duration'] = pd.to_timedelta(agg_df['Time'].astype(str))

# 最终排序并展示
agg_df = agg_df.sort_values(by=['Site_ID', 'Weekday', 'Time_Duration'])
agg_df = agg_df[['Site_ID', 'Weekday', 'Time', 'Aligned_Datetime', 'Bound_Category', 'MeltedValue']]

In [11]:
agg_df

,Site_ID,Weekday,Time,Aligned_Datetime,Bound_Category,MeltedValue
0,10183,0,00:00:00,2023-01-02 00:00:00,All Northbound,0.805825
1,10183,0,00:00:00,2023-01-02 00:00:00,All Southbound,1.155340
2,10183,0,00:15:00,2023-01-02 00:15:00,All Northbound,0.873786
3,10183,0,00:15:00,2023-01-02 00:15:00,All Southbound,0.970874
4,10183,0,00:30:00,2023-01-02 00:30:00,All Northbound,0.679612
...,...,...,...,...,...,...
306331,3130101,6,23:15:00,2023-01-08 23:15:00,All Southbound,187.392857
306332,3130101,6,23:30:00,2023-01-08 23:30:00,All Northbound,160.696429
306333,3130101,6,23:30:00,2023-01-08 23:30:00,All Southbound,171.517857
306334,3130101,6,23:45:00,2023-01-08 23:45:00,All Northbound,148.580357


In [4]:
agg_df.to_csv('mean_processed_triangle.csv', index=False)

# 基本上连续记录的数据

In [4]:
moreDate_data.head()

,Site_ID,Bound_Category,Time,Date,MeltedValue
0,1215468,All Northbound,00:00:00,2023-05-01,1063.0
1,1215468,All Northbound,00:15:00,2023-05-01,936.0
2,1215468,All Northbound,00:30:00,2023-05-01,728.0
3,1215468,All Northbound,00:45:00,2023-05-01,749.0
4,1215468,All Northbound,01:00:00,2023-05-01,661.0


In [9]:
import pandas as pd

# 找出两个 DataFrame 中完全相同的重复行
common_rows = pd.merge(threeDate_data_oneBound, threeDate_data_multipleBound, how='inner')

print("两个 DataFrame 共有的重复行：")
print(common_rows)


两个 DataFrame 共有的重复行：
Empty DataFrame
Columns: [Site_ID, Bound_Category, Time, Date, MeltedValue]
Index: []


# 每年记三天且不止记了一个方向的数据

In [1]:
import pandas as pd
threeDate_data_multipleBound = pd.read_csv("./combined_results_bluedot.csv")

In [2]:
threeDate_data_multipleBound.head()

,Site_ID,Bound_Category,Time,Date,MeltedValue
0,0000261_0154,All Northbound,00:00:00,2023-07-11,NaN
1,0000261_0154,All Northbound,00:15:00,2023-07-11,NaN
2,0000261_0154,All Northbound,00:30:00,2023-07-11,NaN
3,0000261_0154,All Northbound,00:45:00,2023-07-11,NaN
4,0000261_0154,All Northbound,01:00:00,2023-07-11,NaN


In [ ]:
threeDate_data_multipleBound.head()

In [6]:
import pandas as pd

# 假设数据叫 df
df = threeDate_data_multipleBound.copy()

# 转换日期类型
df['Date'] = pd.to_datetime(df['Date'])

# 提取年份和星期几
df['Year'] = df['Date'].dt.year
df['Weekday'] = df['Date'].dt.day_name()

# 按 Site_ID 和 Year 分组，计算唯一 weekday 数量
weekday_counts = df.groupby(['Site_ID', 'Year'])['Weekday'].nunique().reset_index()

# 过滤出 weekday 不满 7 天的记录（即不完整的记录）
incomplete_sites = weekday_counts[weekday_counts['Weekday'] < 7]

In [8]:
incomplete_sites

,Site_ID,Year,Weekday
0,0000001_0105,2024,3
1,0000001_0107,2024,3
2,0000001_0109,2023,3
3,0000001_0112,2024,3
4,0000001_0114,2024,3
...,...,...,...
9118,0000321_8102,2024,3
9119,0000321_8108,2024,3
9120,0000321_8110,2024,3
9121,0000321_8111,2024,3


In [ ]:
# 过滤出MeltedValue缺失的行，打印并检查
missing_meltedvalue = moreDate_data[moreDate_data['MeltedValue'].isna()]

       Site_ID  Bound_Category      Time        Date  MeltedValue
10176   899611  All Southbound  00:00:00  2023-11-14          NaN
10177   899611  All Southbound  00:15:00  2023-11-14          NaN
10178   899611  All Southbound  00:30:00  2023-11-14          NaN
10179   899611  All Southbound  00:45:00  2023-11-14          NaN
10180   899611  All Southbound  01:00:00  2023-11-14          NaN
...        ...             ...       ...         ...          ...
10271   899611  All Southbound  23:45:00  2023-11-14          NaN
10272   899611  All Southbound  00:00:00  2023-11-15          NaN
10273   899611  All Southbound  00:15:00  2023-11-15          NaN
10274   899611  All Southbound  00:30:00  2023-11-15          NaN
10275   899611  All Southbound  00:45:00  2023-11-15          NaN

[100 rows x 5 columns]


In [10]:
# 打印前100行
print(missing_meltedvalue.head(500))

        Site_ID  Bound_Category      Time        Date  MeltedValue
10176    899611  All Southbound  00:00:00  2023-11-14          NaN
10177    899611  All Southbound  00:15:00  2023-11-14          NaN
10178    899611  All Southbound  00:30:00  2023-11-14          NaN
10179    899611  All Southbound  00:45:00  2023-11-14          NaN
10180    899611  All Southbound  01:00:00  2023-11-14          NaN
...         ...             ...       ...         ...          ...
101487  1270312  All Northbound  03:45:00  2024-06-24          NaN
101488  1270312  All Northbound  04:00:00  2024-06-24          NaN
101489  1270312  All Northbound  04:15:00  2024-06-24          NaN
101490  1270312  All Northbound  04:30:00  2024-06-24          NaN
101491  1270312  All Northbound  04:45:00  2024-06-24          NaN

[500 rows x 5 columns]


# 每年记三天且只记了一个方向的数据 这个里面缺的比较多 而且是按照每小时记录的

In [8]:
import pandas as pd
threeDate_data_oneBound = pd.read_csv("./combined_results_failed_bluedot.csv")

In [9]:
threeDate_data_oneBound.head()

,Site_ID,Bound_Category,Time,Date,MeltedValue
0,0000121_R654,All Southbound,00:00:00,2023-02-06,NaN
1,0000121_R654,All Southbound,00:15:00,2023-02-06,NaN
2,0000121_R654,All Southbound,00:30:00,2023-02-06,NaN
3,0000121_R654,All Southbound,00:45:00,2023-02-06,NaN
4,0000121_R654,All Southbound,01:00:00,2023-02-06,NaN


# 处理数据代码

In [10]:
import pandas as pd

def process_dataset(df, id_columns, date_column, time_column, value_column):
    """
    处理任意数据集：
    1. 若整点时间有值，则将整点值除以 4，并填充该小时的 15, 30, 45 分钟的数据。
    2. 若整点时间无值，则删除该小时的所有数据。
    3. 不修改或删除原始 'Date' 和 'Time' 列，而是新增 'Datetime' 进行处理。

    参数：
    - df: pandas DataFrame
    - id_columns: 用于分组的列，如 ['Site_ID', 'Bound_Category']
    - date_column: 日期列名称，如 'Date'
    - time_column: 时间列名称，如 'Time'
    - value_column: 需要填充或删除的列，如 'MeltedValue'
    """
    df = df.copy()

    # 确保 'value_column' 是数值型
    df[value_column] = pd.to_numeric(df[value_column], errors='coerce')

    # 创建新的 'Datetime' 列
    df['Datetime'] = pd.to_datetime(df[date_column].astype(str) + ' ' + df[time_column].astype(str))

    # 提取小时信息
    df['Hour'] = df['Datetime'].dt.hour

    # 处理数据
    def fill_or_remove(group):
        for hour in range(24):
            # 选出当前小时的数据
            hourly_data = group[group['Hour'] == hour]

            # 获取整点数据（例如 00:00:00, 01:00:00, ..., 23:00:00）
            full_hour_record = hourly_data[hourly_data['Datetime'].dt.minute == 0]

            if not full_hour_record.empty and not full_hour_record[value_column].isna().all():
                # 整点有值，将整点值除以 4
                original_value = full_hour_record[value_column].values[0]
                divided_value = original_value / 4

                # 更新整点数据
                group.loc[(group['Hour'] == hour) & (group['Datetime'].dt.minute == 0), value_column] = divided_value

                # 填充 15, 30, 45 分钟的数据
                for minute in [15, 30, 45]:
                    group.loc[(group['Hour'] == hour) & (group['Datetime'].dt.minute == minute), value_column] = divided_value
            else:
                # 整点无值，删除该小时数据
                group = group[group['Hour'] != hour]

        return group

    # 按指定的 ID 列分组，并应用填充/删除逻辑
    processed_data = df.groupby(id_columns, group_keys=False).apply(fill_or_remove)

    # 删除 'Hour' 列并重置索引，但保留 'Datetime'
    processed_data = processed_data.drop(columns=['Hour']).reset_index(drop=True)

    return processed_data

In [11]:
# 示例调用
id_columns = ['Site_ID', 'Bound_Category', 'Date']
time_column = 'Time'
value_column = 'MeltedValue'
date_column = 'Date'

processed_threeDate_data_oneBound = process_dataset(threeDate_data_oneBound, id_columns, date_column,time_column, value_column)

/var/folders/t2/21wy42zs3vqf4r_24_d7j4nc0000gn/T/ipykernel_89522/3213252314.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_data = df.groupby(id_columns, group_keys=False).apply(fill_or_remove)


In [13]:
processed_threeDate_data_oneBound.to_csv('processed_threeDate_data_oneBound.csv', index=False)

In [27]:
processed_threeDate_data_oneBound.head()

,Site_ID,Bound_Category,Time,Date,MeltedValue,Datetime
0,0000189_8033,All Northbound,18:00:00,2024-06-25,2.00,2024-06-25 18:00:00
1,0000189_8033,All Northbound,18:15:00,2024-06-25,2.00,2024-06-25 18:15:00
2,0000189_8033,All Northbound,18:30:00,2024-06-25,2.00,2024-06-25 18:30:00
3,0000189_8033,All Northbound,18:45:00,2024-06-25,2.00,2024-06-25 18:45:00
4,0000189_8033,All Northbound,19:00:00,2024-06-25,1.25,2024-06-25 19:00:00


In [6]:
id_columns = ['Site_ID', 'Bound_Category', 'Date']
time_column = 'Time'
value_column = 'MeltedValue'
date_column = 'Date'
processed_subset = process_dataset(subset, id_columns, date_column,time_column, value_column)

/var/folders/t2/21wy42zs3vqf4r_24_d7j4nc0000gn/T/ipykernel_59858/3213252314.py:55: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  processed_data = df.groupby(id_columns, group_keys=False).apply(fill_or_remove)


In [ ]:
processed_subset

# 合并数据

In [14]:
import pandas as pd

moreDate_data = pd.read_csv("./combined_results_deduplicated_triangle.csv")

# 读取你之前处理过的单一方向数据
processed_threeDate_data_oneBound = pd.read_csv('processed_threeDate_data_oneBound.csv')

# 读取原始的 blue dot 数据
combined_results_bluedot = pd.read_csv('combined_results_bluedot.csv')

In [15]:
moreDate_data.head(1)

,Site_ID,Bound_Category,Time,Date,MeltedValue
0,1215468,All Northbound,00:00:00,2023-05-01,1063.0


In [16]:
processed_threeDate_data_oneBound.head(1)
processed_threeDate_data_oneBound = processed_threeDate_data_oneBound[['Site_ID', 'Bound_Category', 'Time', 'Date', 'MeltedValue']]

In [9]:
combined_results_bluedot.head(1)

,Site_ID,Bound_Category,Time,Date,MeltedValue
0,0000261_0154,All Northbound,00:00:00,2023-07-11,NaN


In [17]:
combined_df = pd.concat([moreDate_data, processed_threeDate_data_oneBound, combined_results_bluedot], ignore_index=True)

In [18]:
# 查看合并后的结果
print(combined_df.shape)
print(combined_df.columns)

(37628832, 5)
Index(['Site_ID', 'Bound_Category', 'Time', 'Date', 'MeltedValue'], dtype='object')


In [19]:
combined_df.to_csv('all_data_combined.csv', index=False)